In [ ]:
%matplotlib notebook
from pylab import *
from IPython.display import display, Latex
import ipywidgets as widgets

In [ ]:
flines = open('nuclide masses.txt', 'r').readlines()

In [ ]:
# find lines starting a new element
elines = []
for n, line in enumerate(flines):
    words = line.split('\t')
    if words[0].isdigit():
        elines.append(n)

In [ ]:
def extractFloat(sVal):
    '''utility function to remove "(" from 
       floating point numbers, if present'''
    if len(sVal)>0:
        try:
            ind = sVal.index('(')
            sVal = sVal[:ind]
        except ValueError:
            pass
        return float(sVal)
    else:
        return ""

def extractElement(start = 8, end = 15):
    '''Grab data for element with starting line and ending line'''
    words = flines[start].split('\t')
    Z = int(words[0])
    A = [ int(words[2]) ]
    m = [ extractFloat(words[3])]
    if Z < 3:
        abundance = [ extractFloat(words[4]) ]
    else:
        abundance = [ '' ]
    symbol = words[1]
    for line in flines[start+1: end-1]:
        words = line.split('\t')
        A.append(int(words[1]))
        m.append(extractFloat(words[2]))
        abund = '' if Z >= 83 else extractFloat(words[3])
        abundance.append(abund)
        
    return Z, symbol, A, m, abundance


In [ ]:
# This cell generates the dictionary of nuclides
nuclides = {}
for start, end  in list(zip(elines, elines[1:])):
    Z, symbol, A, m, abundance = extractElement(start, end)
    nuclides[symbol] = {}
    ns = nuclides[symbol]
    ns['Z'] = Z
    ns['A'] = A
    ns['m'] = m
    ns['abundance'] = abundance
    
    
    
    

In [ ]:
# Create some widgets:
drop_layout = widgets.Layout(width = '75px')
check_layout = widgets.Layout(width = '200px')
label = widgets.HTML(value ="<font color=blue><b>Select Element:   </b></font>" )
options = ["None"] + list(nuclides.keys())
drop_element = widgets.Dropdown(options = options, 
                               layout = drop_layout)
check1 = widgets.Checkbox(value=False, description = "Show 1:1 line", layout = check_layout)
check2 = widgets.Checkbox(value=False, description = "Show 2:1 line", layout = check_layout)

In [ ]:
# define plotting function and arrange widgets
fig = figure("Nuclear Stability")
outbox = widgets.Output(layout = {'border': '1px solid blcak'})
display(outbox)

def plotNuc(zz = -1, show1 = False, show2 = False):
    '''zz gives Z for putting red borders on dots,
       show1: determines whether to show 1:1 line,
       show2:  determines whether to show 2:1 line'''
    ax = fig.gca()
    ax.cla()
    for v in nuclides.values():
        Z = v['Z']
        a = array(v['A'])
        z = array([Z]*len(a))
        n = a-z
        not_stable = array(v['abundance']) == ''
        #edge = 'red' if Z==20 else 'cyan'
        edge = 'cyan'
        scatter(z[not_stable], n[not_stable], color=edge, facecolor='none', alpha = 0.5, s=20)

    for v in nuclides.values():
        Z = v['Z']
        a = array(v['A'])
        z = array([Z]*len(a))
        n = a-z
        is_stable = array(v['abundance']) != ''
        colors = where(is_stable, '#000000', '#00FFFF')
        size = where(is_stable, 50, 30)
        edge = 'red' if Z==zz else 'black'
        # stable:
        scatter(z[is_stable], n[is_stable], color='black', edgecolor=edge, s = 50)
    
    if show1:
        plot([0,120], [0, 120], 'g-')
    if show2:
        plot([0,85], [0, 175], 'b-')
    grid()
    title("Belt of Stability")
    xlabel('Z')
    ylabel('# neutrons');
    
plotNuc()

def select_data(change):
    outbox.clear_output(wait=True)
    plotNuc(zz = drop_element.index - 1, show1 = check1.value, show2 = check2.value)
    element = drop_element.value
    if element != "None":
        ne = nuclides[element]
        is_stable = array(ne['abundance']) != ''
        Alist = [a for a in ne['A']*is_stable if a]
        sForm = r"$^{%s}{%s},\:$ " % ("%d", element)
        sList = sForm * (len(Alist)-1) + sForm[:-5] + '$'  # end without comma
        sList = sList % tuple(Alist)
        output = "Stable Isotopes of %s:  %s" % (drop_element.value, sList)
        with outbox:
            display(Latex(output))
    else:
        outbox.clear_output()
    
drop_element.observe(select_data, names='value')
check1.observe(select_data, names = 'value')
check2.observe(select_data, names = 'value')

widgets.HBox([label, drop_element, check1, check2])


In [ ]:
%%html
<style>
div.input{
    display:none;
}
</style>